In [1]:
# DigitRecognizer LeNet5 v1
#
# CNN: LetNet5 v1
#  - Relu
#  - MaxPooling
#  - Softmac
#
# Kaggle Scoring: 0.98885, Position #750 (17/1/2018)
#
# Seed: 1
# Epochs: 165
# Batch Size: 32
# Train metrics: {'global_step': 173250, 'recall': 1.0, 'precision': 1.0, 'f1_score': 1.0, 'loss': 0.0, 'accuracy': 1.0}
# Val metrics: {'global_step': 173250, 'recall': 0.9996025, 'precision': 0.99933767, 'f1_score': 0.99947006736346533, 'loss': 0.19570193, 'accuracy': 0.99000001}

In [2]:
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split

In [12]:
SEED = 1

BATCH_SIZE = 32   #33.600 / 32 = 1050 globalstep / epoch => 1050 * 165 = 173.250 global_step
EPOCHS = 165

In [4]:
data_train = np.genfromtxt("train.csv", delimiter=',', dtype=np.float32)
data_test  = np.genfromtxt("test.csv",  delimiter=',', dtype=np.float32)

In [5]:
#label,pixel0,pixel1,pixel2,..,pixel783
X_train = data_train[1:,1:]   #(42000, 784)
y_train = data_train[1:,0:1]  #(42000, 1)
X_test  = data_test[1:,:]     #(28000, 784)

print("X_train.shape: {}".format(X_train.shape))
print("X_test.shape: {}".format(X_test.shape))

X_train.shape: (42000, 784)
X_test.shape: (28000, 784)


In [6]:
X_train = X_train.reshape((-1,28,28,1)) #(42000, 28, 28, 1)
X_test  = X_test.reshape((-1,28,28,1)) #(42000, 28, 28, 1)


X_train = X_train / 255.0
X_test  = X_test / 255.0

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.20, random_state=SEED)

print("X_train.shape: {}".format(X_train.shape))  #(33600, 28, 28, 1)
print("X_val.shape: {}".format(X_val.shape))      #(8400, 28, 28, 1)
print("X_test.shape: {}".format(X_test.shape))    #(28000, 28, 28, 1)

X_train.shape: (33600, 28, 28, 1)
X_val.shape: (8400, 28, 28, 1)
X_test.shape: (28000, 28, 28, 1)


In [7]:
# Model: Lenet-5 variation (max_pooling, relu and SAME padding)

def cnn_model_fn( features, labels, mode):
    # Conv1: 14x14@6
    model = tf.layers.conv2d( 
        inputs = features["x"], 
        filters = 6, 
        kernel_size = [5, 5],
        strides = 1, 
        padding = "SAME",
        activation = tf.nn.relu)
    model = tf.layers.max_pooling2d( 
        inputs = model,
        pool_size = [ 2, 2],
        strides = 2)
    
    # Conv2: 7x7@16
    model = tf.layers.conv2d( 
        inputs = model, 
        filters = 16, 
        kernel_size = [5, 5],
        strides = 1, 
        padding = "SAME",
        activation = tf.nn.relu)
    model = tf.layers.max_pooling2d( 
        inputs = model,
        pool_size = [ 2, 2],
        strides = 2)
    
    model = tf.layers.flatten( inputs = model) # 784 nodes
    
    # FC
    model = tf.layers.dense( 
        inputs = model,
        units = 120,
        activation = tf.nn.relu)

    model = tf.layers.dense( 
        inputs = model,
        units = 84,
        activation = tf.nn.relu)

    # Output (softmax)
    logits = tf.layers.dense( model, 10, name = "output_tensor")
    
    predictions = {
        "classes": tf.argmax( logits, axis=1),
        "probabilities": tf.nn.softmax( logits, name="softmax_tensor")
    }
    
    
    # PREDICT
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode = mode, predictions = predictions)
    
    
    # TRAIN
    onehot_labels = tf.one_hot( indices = tf.cast(labels, tf.int32), depth = 10)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits( 
        logits=logits, 
        labels=onehot_labels), name="cost_tensor")
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        train_op = tf.train.AdamOptimizer().minimize( cost, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode = mode, loss = cost, train_op = train_op)
    
    
    # EVAL
    eval_metrics_ops = {
        "accuracy": tf.metrics.accuracy(
            labels = labels, 
            predictions = predictions["classes"]),
        
        "precision": tf.metrics.precision(
            labels = labels,
            predictions = predictions["classes"]),
        
        "recall": tf.metrics.recall(
            labels = labels,
            predictions = predictions["classes"])
    }
    return tf.estimator.EstimatorSpec( mode = mode, loss = cost, eval_metric_ops = eval_metrics_ops)

In [8]:
def train_model( X_train, y_train, X_val, y_val):
    def f1_score( precision, recall):
        return 2.0 / (precision**-1 + recall**-1)
    
    model = tf.estimator.Estimator(
        model_fn = cnn_model_fn,
        model_dir="/tmp/cnn_lenet5_v1_digits_model")

    model.train(
        input_fn = tf.estimator.inputs.numpy_input_fn(
            x = {"x": X_train},
            y = y_train,
            batch_size = BATCH_SIZE,
            num_epochs = EPOCHS,
            shuffle = True))

    # Model performance
    train_metrics = model.evaluate(
        input_fn = tf.estimator.inputs.numpy_input_fn(
            x = {"x": X_train},
            y = y_train,
            batch_size = BATCH_SIZE,
            num_epochs = 1,
            shuffle = False))

    val_metrics = model.evaluate(
        input_fn = tf.estimator.inputs.numpy_input_fn(
            x = {"x": X_val},
            y = y_val,
            batch_size = BATCH_SIZE,
            num_epochs = 1,
            shuffle = False))
    
    train_metrics["f1_score"] = f1_score( train_metrics["precision"], train_metrics["recall"])
    val_metrics["f1_score"]   = f1_score( val_metrics["precision"], val_metrics["recall"])

    print(">> Train metrics: %r"% train_metrics)
    print(">> Val metrics: %r"% val_metrics)
    
    return model, train_metrics, val_metrics

In [9]:
def predict_digits( model, X_test):
    predictions = model.predict(
        input_fn = tf.estimator.inputs.numpy_input_fn(
            x = {"x": X_test},
            num_epochs = 1,
            shuffle = False))
    
    csv = open( "digit_prediction.csv", "w")
    csv.write("ImageId,Label\n")
    for i, pred in enumerate(predictions):
        csv.write("{},{}\n".format( i+1, pred["classes"]))
    csv.close()

In [13]:
tf.logging.set_verbosity(tf.logging.ERROR)

with tf.Session() as sess:
    tf.set_random_seed( SEED)
    model, _, _ = train_model( X_train, y_train, X_val, y_val)
    predict_digits( model, X_test)

>> Train metrics: {'global_step': 173250, 'recall': 1.0, 'precision': 1.0, 'f1_score': 1.0, 'loss': 0.0, 'accuracy': 1.0}
>> Val metrics: {'global_step': 173250, 'recall': 0.9996025, 'precision': 0.99933767, 'f1_score': 0.99947006736346533, 'loss': 0.19570193, 'accuracy': 0.99000001}
